## Hadi Heidari Rad - 810197011

### AI - CA4

In [85]:
import numpy as np
import pandas as pd
from sklearn import tree
import random
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz

data = pd.read_csv('data.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


### Part 1

First, shuffling the data and then allocating 0.8 of it to train data and the rest 0.2 to test data:

In [86]:
# shuffling dataset:
data = data.sample(frac = 1)

separated_data = np.split(data, [round(len(data) * 0.8)], axis=0)
train_data_x = separated_data[0].drop(columns=['target'])
train_data_y = separated_data[0]['target']

test_data_x = separated_data[1].drop(columns=['target'])
test_data_y = separated_data[1]['target']

Using the train data to predict the test one with decision tree and then calcualting accuracy:

In [87]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_data_x, train_data_y)
result = clf.predict(test_data_x)

print("accuracy: " + str(accuracy_score(test_data_y, result)))

accuracy: 0.7377049180327869


### Part 2.1 and 2.2

Here we make 5 trees with 150 samples of train data and then save the predicted list somewhere, at last for each test data element, final predicted value will be the value which is most predicted by those 5 trees.

In [65]:
res = []
final = []
num_of_trees = 5
for i in range (0, num_of_trees):
    bagging_data = separated_data[0].sample(n = 150)
    bagging_train_x = bagging_data.drop(columns=['target'])
    bagging_train_y = bagging_data['target']
    
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(bagging_train_x, bagging_train_y)
    result = clf.predict(test_data_x)
    res.append(result)


for i in range (0, len(res[0])):
    sum = 0
    for j in range(0, len(res)):
        sum += res[j][i]
    if sum > (num_of_trees/2):
        final.append(1)
    else:
        final.append(0)

print("accuracy: " + str(accuracy_score(test_data_y, final)))

accuracy: 0.8032786885245902


### part 2.3

Deleting columns one by one and then making the decision tree:

In [77]:

for (columnName, columnData) in train_data_x.iteritems():
    train_without_one_col = train_data_x.drop(columns=[columnName])
    test_without_one_col = test_data_x.drop(columns=[columnName])
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(train_without_one_col, train_data_y)
    result = clf.predict(test_without_one_col)

    print("accuracy without "+ str(columnName)+ ": "+ str(accuracy_score(test_data_y, result)))
    

accuracy without age: 0.7540983606557377
accuracy without sex: 0.7213114754098361
accuracy without cp: 0.639344262295082
accuracy without trestbps: 0.6721311475409836
accuracy without chol: 0.7049180327868853
accuracy without fbs: 0.7213114754098361
accuracy without restecg: 0.6721311475409836
accuracy without thalach: 0.7049180327868853
accuracy without exang: 0.7213114754098361
accuracy without oldpeak: 0.7213114754098361
accuracy without slope: 0.7704918032786885
accuracy without ca: 0.639344262295082
accuracy without thal: 0.7377049180327869


### part 2.4

choosing 5 features among 13 (by random) and calculating the accuracy:

In [26]:
def split_five_features(train_x, train_y, test_x, test_y):

    for i in range(0, 8):
        rnd = random.randrange(0, len(train_x.columns))
        train_x = train_x.drop(train_x.columns[rnd], axis = 1)
        test_x = test_x.drop(test_x.columns[rnd], axis = 1)

    # print(five_train)
    # print(five_test)

    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(train_x, train_y)
    result = clf.predict(test_x)
    return result

ans = accuracy_score(test_data_y, split_five_features(train_data_x, train_data_y, test_data_x, test_data_y))
print("accuracy: " + str(ans) )

accuracy: 0.7868852459016393


### part 2.5

Now the goal is to implement bagging but with a difference: features are chosen randomized too (5 here) and this is called a random forest.

In [73]:
res = []
final = []
num_of_trees = 100
for i in range (0, num_of_trees): # 100 trees
    bagging_data = separated_data[0].sample(n = 150)
    bagging_train_y = bagging_data['target']
    bagging_train_x = bagging_data.drop(columns=['target'])
    
    result = split_five_features(bagging_train_x, bagging_train_y, test_data_x, test_data_y)
    res.append(result)


for i in range (0, len(res[0])):
    sum = 0
    for j in range(0, len(res) - 1):
        sum += res[j][i]
    if sum > (num_of_trees/2):
        final.append(1)
    else:
        final.append(0)

print("accuracy: " + str(accuracy_score(test_data_y, final)))

accuracy: 0.819672131147541


<br>
Boostrapping aggregation or bagging is when we choose to build more than one tree but with a fraction of sample numbers (compared to the original train data). these trees' train datas can have overlapping samples. Then each tree separately gives a list of prediction, and the final prediction for each test data sample is determined by the target that is most predicted. boostrapping also reduces the variance and standard deviation while retaining the bias because it frustrates effect of noise samples.
<br><br>
Overfitting is when our data analysis model is very fitted to the observed (or trained) data and cannot fit or predict new data well, and accurate. In a decision tree, over-fitting happens so likely because the tree is generated in a way to perfectly fit all samples in the training data set. So this effects the accuracy when predicting samples that are new. Bagging reduces overfitting. and the the reason is obvious, instead of fitting the model on just one sample of the train set, several models are fitted on different samples.
<br><br>
They are exactly the same except in one thing: In a random forest we do randomly select features too. But in Bagging all features are selected. So, a random forest is a bagging with also a feature bagging. Feature bagging (or Random Suspace Method) helps the model to escape overfitting on features, and we can combine fittings of several models and reduce the effect of some highly predictive or descriptive features overfitting.
<br><br>
There is an important point here!! <br>
While the algorithms used here are very randomized, results are different each time running. And there were no information given about whether it is needed to consider one or more specific seed (random state) for decision trees in the project definition. So I didn't fix a random state. And results are different each time. 
<br>
But clearly, 2.2 one works better on unfamiliar data than 1, and 2.5 one works better in all aspects(!), than part 2.2